In [1]:
import jax.numpy as jnp
from jax import random
import pandas as pd
from tqdm.notebook import tqdm
import corner
import numpyro
from numpyro import distributions as dst
from numpyro.infer import NUTS,MCMC,MixedHMC
choco = pd.read_csv('chocolate.csv')
import jax
# Set the hyperparameters
prior_n = 20
prior_m = 0.9
nu = 2
mu = 4
s = 1
r = 0.1

# Set the observed data
N = len(choco) # number of tested chocolate bars
X = choco['Rating'] >= 3 # number of tasty bars (maybe you can tweak it and set your expectations higher?)

2023-01-04 15:05:32.832328: W external/org_tensorflow/tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/x86_64-linux-gnu/:/opt/Zotero:/opt/AMD/aocc-compiler-4.0.0/bin:/opt/AMD/aocc-compiler-4.0.0/share/opt-viewer:/usr/local/texlive/2022/bin/x86_64-linux:/home/mitra/.local/bin:/home/mitra/.opam/default/bin:/home/mitra/.cargo/bin:/home/mitra/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/snap/bin:/home/mitra/.local/share/coursier/bin:/usr/local/pgplot/
2023-01-04 15:05:32.954845: W external/org_tensorflow/tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/x86_64-linux-gnu/:/opt/Zotero:/opt/AMD/aocc-com

In [2]:
def model(data,N,prior_n,prior_m,nu,mu,s,r):
    p_n=numpyro.param("prior n",prior_n)
    p_m=numpyro.param("prior m",prior_m)
    nu=numpyro.param("nu",nu)
    mu=numpyro.param("mu",mu)
    s=numpyro.param("s",s)
    r=numpyro.param("r",r)
    m=numpyro.sample("m",dst.Beta(mu,nu))
    n=numpyro.sample("n",dst.Gamma(s,r))
    theta=numpyro.sample("theta",dst.Beta(m*n,m*(n-1)))
    with numpyro.plate("data",N):
        numpyro.sample("obs",dst.Binomial(probs=theta),obs=data)
    

In [ ]:
rng_key = random.PRNGKey(0)
rng_key, rng_key_ = random.split(rng_key)

# Run NUTS.
kernel = NUTS(model,step_size=0.1)
num_samples = 35000
mcmc = MCMC(kernel, num_warmup=5000, num_samples=num_samples)
mcmc.run(
    rng_key_, data=jnp.array(X),N=N,prior_m=prior_m,prior_n=prior_n,nu=nu,mu=mu,s=s,r=r
)
mcmc.print_summary()
samples_1 = mcmc.get_samples()

In [ ]:
samples_1